In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import b3d
import sys

sys.path.append(str(b3d.get_root_path()))

In [3]:
from tests.sama4d.video_to_tracks.video_to_tracks_task import VideoToTracksTask

In [4]:
path = b3d.get_assets_path() / 'shared_data_bucket/dynamic_SfM/feature_track_data/pan_around_blocks.npz'
ftd_og = b3d.io.FeatureTrackData.load(str(path)).slice_time(20, 35)

In [5]:
import rerun
rerun.init("multiple_patch_tracking-3")
rerun.connect("127.0.0.1:8812")

In [6]:
task = VideoToTracksTask.from_feature_track_data(ftd_og, has_no_moving_objects=True)
task.visualize_task()

In [20]:
from tests.sama4d.video_to_tracks.solvers.conv_from_grid import KeypointTracker2DFromInitialGrid

spec = task.get_task_specification()
solver = KeypointTracker2DFromInitialGrid(
    grid_size_x=12, grid_size_y=12, patch_size=9, do_reinitialization=True
)
solution = solver.solve(spec, save_states=True)

In [21]:
solver.visualize_solver_state()

In [23]:
solver.update_blueprint(31)